In [27]:
import pandas as pd

In [61]:
df_movimientos = pd.read_excel('input\\Formato Movimientos - Inventario.xlsx')
df_movimientos['NOMBRE'] = None
df_movimientos.rename(columns = {'Codigo Articulo ': 'CODIGO', 'Lote': 'LOTE', 'Fecha Vencimiento': 'FECHA VENCIMIENTO', 'Numero Serie': 'SERIE', 'Cantidad Movidad': 'CANTIDAD'}, inplace = True)
df_movimientos = df_movimientos[['CODIGO', 'NOMBRE', 'CANTIDAD', 'SERIE', 'LOTE', 'FECHA VENCIMIENTO', 'Tipo Movimiento', 'Fecha Movimiento']]

df_inventario_malo = pd.read_excel('input\\Inventario_A_malo.xlsx')
df_inventario_malo.rename(columns =  {'NOMBRE ARTICULO': 'NOMBRE', 'STOCK': 'CANTIDAD'}, inplace = True)

df_inventario_bueno = pd.read_excel('input\\Inventario_B_bueno.xlsx')

# Antes de comparar y ver si están las llaves, CODIGO, LOTE y FECHA DE VENCIMIENTO DEBEN ESTAR EN FORMATOS IGUALES.
df_movimientos['CODIGO'] = df_movimientos['CODIGO'].astype(str).apply(lambda x: x.strip())
df_inventario_malo['CODIGO'] = df_inventario_malo['CODIGO'].astype(str).apply(lambda x: x.strip())
df_inventario_bueno['CODIGO'] = df_inventario_bueno['CODIGO'].astype(str).apply(lambda x: x.strip())

df_movimientos['LOTE'] = df_movimientos['LOTE'].astype(str).apply(lambda x: x.strip())
df_inventario_malo['LOTE'] = df_inventario_malo['LOTE'].astype(str).apply(lambda x: x.strip())
df_inventario_bueno['LOTE'] = df_inventario_bueno['LOTE'].astype(str).apply(lambda x: x.strip())

df_movimientos['FECHA VENCIMIENTO'] = pd.to_datetime(df_movimientos['FECHA VENCIMIENTO'])
df_inventario_bueno['FECHA VENCIMIENTO'] = pd.to_datetime(df_inventario_bueno['FECHA VENCIMIENTO'], errors = 'coerce')
df_inventario_malo['FECHA VENCIMIENTO'] = pd.to_datetime(df_inventario_malo['FECHA VENCIMIENTO'], errors = 'coerce')

In [30]:
agrupador = ['CODIGO', 'LOTE', 'FECHA VENCIMIENTO']
movimientos_agrupados = df_movimientos.groupby(by = agrupador).sum()
inventario_malo_agrupado = df_inventario_malo.groupby(by = agrupador).sum()
inventario_bueno_agrupado = df_inventario_bueno.groupby(by = agrupador).sum()

In [31]:
# Primero, quiero ver las diferencias entre códigos que hay entre el inventario malo y el bueno.

movimientos_agrupados = df_movimientos.groupby(by = 'CODIGO').sum()
inventario_malo_agrupado = df_inventario_malo.groupby(by = 'CODIGO').sum()
inventario_bueno_agrupado = df_inventario_bueno.groupby(by = 'CODIGO').sum()

diferencias = pd.merge(inventario_malo_agrupado, inventario_bueno_agrupado, how = 'outer', left_index = True, right_index = True, indicator = True)
diferencias = diferencias[diferencias['_merge'] != 'both']

print(f'Hay {diferencias.shape[0]} códigos distintos entre el inventario bueno y malo')

diferencias.to_excel('Diferencias_inventario_codigo.xlsx')

# Segundo, quiero ver las diferencias entre códigos/lote que hay entre el inventario malo y el bueno.

movimientos_agrupados = df_movimientos.groupby(by = ['CODIGO', 'LOTE']).sum()
inventario_malo_agrupado = df_inventario_malo.groupby(by = ['CODIGO', 'LOTE']).sum()
inventario_bueno_agrupado = df_inventario_bueno.groupby(by = ['CODIGO', 'LOTE']).sum()

diferencias = pd.merge(inventario_malo_agrupado, inventario_bueno_agrupado, how = 'outer', left_index = True, right_index = True, indicator = True)
diferencias = diferencias[diferencias['_merge'] != 'both']

print(f'Hay {diferencias.shape[0]} códigos/lotes distintos entre el inventario bueno y malo')

diferencias.to_excel('Diferencias_inventario_codigo_lote.xlsx')

# Tercero, quiero ver las diferencias entre códigos/lote/fecha vencimiento que hay entre el inventario malo y el bueno.

movimientos_agrupados = df_movimientos.groupby(by = ['CODIGO', 'LOTE', 'FECHA VENCIMIENTO']).sum()
inventario_malo_agrupado = df_inventario_malo.groupby(by = ['CODIGO', 'LOTE', 'FECHA VENCIMIENTO']).sum()
inventario_bueno_agrupado = df_inventario_bueno.groupby(by = ['CODIGO', 'LOTE', 'FECHA VENCIMIENTO']).sum()

diferencias = pd.merge(inventario_malo_agrupado, inventario_bueno_agrupado, how = 'outer', left_index = True, right_index = True, indicator = True)
diferencias = diferencias[diferencias['_merge'] != 'both']

print(f'Hay {diferencias.shape[0]} códigos/lotes/fecha distintos entre el inventario bueno y malo')

diferencias.to_excel('Diferencias_inventario_codigo_lote_fecha.xlsx')

Hay 176 códigos distintos entre el inventario bueno y malo
Hay 395 códigos/lotes distintos entre el inventario bueno y malo
Hay 341 códigos/lotes/fecha distintos entre el inventario bueno y malo


In [22]:
# Ahora, hay que ver la diferencia entre los movimientos e inventario "malo".

movimientos_agrupados = df_movimientos.groupby(by = ['CODIGO']).sum()
inventario_malo_agrupado = df_inventario_malo.groupby(by = ['CODIGO']).sum()

diferencias = pd.merge(movimientos_agrupados, inventario_malo_agrupado, how = 'outer', left_index = True, right_index = True, indicator = True)
diferencias = diferencias[diferencias['_merge'] == 'left_only']

print(f'Hay {diferencias.shape[0]} códigos que sólo están en los movimientos, y no en el inventario malo')
diferencias.to_excel('Diferencias_movimientos_inventario_malo_codigo.xlsx')

# Segundo, quiero ver las diferencias entre códigos/lote que hay entre el inventario malo y el bueno.

movimientos_agrupados = df_movimientos.groupby(by = ['CODIGO', 'LOTE']).sum()
inventario_malo_agrupado = df_inventario_malo.groupby(by = ['CODIGO', 'LOTE']).sum()

diferencias = pd.merge(movimientos_agrupados, inventario_malo_agrupado, how = 'outer', left_index = True, right_index = True, indicator = True)
diferencias = diferencias[diferencias['_merge'] == 'left_only']

print(f'Hay {diferencias.shape[0]} códigos/lotes distintos entre el inventario bueno y malo')

diferencias.to_excel('Diferencias_movimientos_inventario_malo_codigo_lote.xlsx')

# Tercero, quiero ver las diferencias entre códigos/lote/fecha que hay entre el inventario malo y el bueno.

movimientos_agrupados = df_movimientos.groupby(by = ['CODIGO', 'LOTE', 'FECHA VENCIMIENTO']).sum()
inventario_malo_agrupado = df_inventario_malo.groupby(by = ['CODIGO', 'LOTE', 'FECHA VENCIMIENTO']).sum()

diferencias = pd.merge(movimientos_agrupados, inventario_malo_agrupado, how = 'outer', left_index = True, right_index = True, indicator = True)
diferencias = diferencias[diferencias['_merge'] == 'left_only']

print(f'Hay {diferencias.shape[0]} códigos/lotes/fecha distintos entre el inventario bueno y malo')

diferencias.to_excel('Diferencias_movimientos_inventario_malo_codigo_lote_fecha.xlsx')

Hay 203 códigos que sólo están en los movimientos, y no en el inventario malo


In [89]:
df_entradas = df_movimientos[df_movimientos['Tipo Movimiento'] == 'ENTRADA']
df_salidas = df_movimientos[df_movimientos['Tipo Movimiento'] == 'SALIDA']

In [90]:
inventario_inicial = df_inventario_bueno.groupby(by = ['CODIGO', 'LOTE']).sum()
movimientos_entrada = df_entradas.groupby(by = ['CODIGO', 'LOTE']).sum()
movimientos_salida = df_salidas.groupby(by = ['CODIGO', 'LOTE']).sum()

In [91]:
# Primero se suman las entradas al inventario
inventario_mas_entradas = inventario_inicial.add(movimientos_entrada, fill_value = 0)
# luego, se restan las salidas al inventario
inventario_total = inventario_mas_entradas.sub(movimientos_salida, fill_value = 0)

In [92]:
with pd.ExcelWriter('resumen_inventario.xlsx', 'openpyxl') as writer:
    inventario_total.to_excel(writer, sheet_name = 'Inventario_total')
    inventario_inicial.to_excel(writer, sheet_name = 'Inventario_inicial')
    movimientos_entrada.to_excel(writer, sheet_name = 'Movimientos_entrada')
    movimientos_salida.to_excel(writer, sheet_name = 'Movimientos_salida')
    